In [1]:
# Import data

In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import numpy as np


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <FB2FD416-6C4D-3621-B677-61F07C02A3C5> /opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/spatialhackathon/lib/python3.9/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/spatialhackathon/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might b

In [4]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch

# 載入資料
train_image_data = torch.load("../SML_train_dataset.pt")
graph_data = torch.load("../train_graph_dataset.pt")

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_30582/2116005886.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_image_data = torch.load("../SML_train_d

In [9]:
import torch
from torch.utils.data import Dataset

class importDataset(Dataset):
    def __init__(self, S_tiles, M_tiles, L_tiles, labels, meta_info, normal_coords, node_feats, adj_lists,edge_feats):
        """
        Custom Dataset to load image tiles, labels, and edge indices for training.
        
        Args:
            S_tiles (list): Small scale image tiles (spot-level features).
            M_tiles (list): Medium scale image tiles (spot-level features).
            L_tiles (list): Large scale image tiles (spot-level features).
            labels (list): Ground truth labels (spot-level cell type compositions).
            meta_info (list): Metadata containing slide_id, x, y coordinates for each spot.
            normal_coords (list): Normalized coordinates for each spot.
            slide_edge_indices (dict): Dictionary containing the slide-level edge indices for each slide.
        """
        self.S_tiles = S_tiles
        self.M_tiles = M_tiles
        self.L_tiles = L_tiles
        self.labels = labels
        self.meta_info = meta_info
        self.normal_coords = normal_coords
        self.node_feats = node_feats
        self.adj_lists = adj_lists
        self.edge_feats = edge_feats
        # Optional: Apply any additional processing to the data here
        # e.g., scaling, normalization, etc.

    def __len__(self):
        # Return the number of samples (spots) in the dataset
        return len(self.S_tiles)

    def __getitem__(self, idx):
        # Get the spot-level data for the sample at index `idx`
        S_tile = self.S_tiles[idx]
        M_tile = self.M_tiles[idx]
        L_tile = self.L_tiles[idx]
        label = self.labels[idx]
        meta = self.meta_info[idx]  # This is a tuple (slide_id, x, y)
        normal_coord = self.normal_coords[idx]
        adj_list = self.adj_lists[idx]
        
        # Convert the tiles to the correct format (channels-first)
        S_tile = torch.tensor(S_tile, dtype=torch.float).permute(2, 0, 1)  # Convert from (H, W, 3) to (3, H, W)
        M_tile = torch.tensor(M_tile, dtype=torch.float).permute(2, 0, 1)
        L_tile = torch.tensor(L_tile, dtype=torch.float).permute(2, 0, 1)
        
        # Convert label and normal_coord to tensors
        label = torch.tensor(label, dtype=torch.float)
        normal_coord = torch.tensor(normal_coord, dtype=torch.float)
        # ➕ Graph features
        max_neighbors = 7  # or whatever k you use
        adj_list = self.adj_lists[idx] if self.adj_lists is not None else []

        # 轉成固定長度的 Tensor
        adj_array = np.zeros((max_neighbors, 2))
        for i, (j, w) in enumerate(adj_list[:max_neighbors]):
            adj_array[i] = [j, w]
         # 🛠️ edge_feat 轉型處理
        edge_feat_i = self.edge_feats[idx]
        if isinstance(edge_feat_i, np.ndarray) and edge_feat_i.dtype == object:
            edge_feat_i = np.array(edge_feat_i.tolist())
        elif isinstance(edge_feat_i, list):
            edge_feat_i = np.array(edge_feat_i)


        graph_feats = {
            'node_feat': torch.tensor(self.node_feats[idx], dtype=torch.float) if self.node_feats is not None else None,
            'adj_list': torch.tensor(adj_array, dtype=torch.float),
            'edge_feat': torch.tensor(edge_feat_i, dtype=torch.float) if edge_feat_i is not None else None,
        }
        # Return the data in a PyTorch-friendly format as a dictionary
        return {
            'S_tile': S_tile,
            'M_tile': M_tile,
            'L_tile': L_tile,
            'label': label,
            'meta': meta,
            'normal_coord': normal_coord,
            **graph_feats  # ➕ 合併進 batch dictionary
        }


In [10]:
# ====== Step 2: 建立 Dataset ======
image_train_dataset = importDataset(
    S_tiles=train_image_data['S_tiles'],
    M_tiles=train_image_data['M_tiles'],
    L_tiles=train_image_data['L_tiles'],
    labels=train_image_data['labels'],
    meta_info=train_image_data['meta_info'],
    normal_coords=train_image_data['normal_coords'],
    node_feats=graph_data['node_feats'],
    adj_lists=graph_data['adj_lists'],
    edge_feats=graph_data['edge_feats']
)


In [14]:
def check_dataset_item(dataset, idx=0):
    item = dataset[idx]

    print("🔍 Checking dataset sample:", idx)

    # --- Center Tile ---
    tile = item['S_tile']
    print(f"📏 Center tile shape: {tile.shape} | dtype: {tile.dtype} | min: {tile.min():.3f}, max: {tile.max():.3f}")

    # --- Subtiles ---
    subtiles = item['M_tile']
    print(f"📏 Subtiles shape: {subtiles.shape} | dtype: {subtiles.dtype}")

    # --- Neighbors ---
    neighbors = item['L_tile']
    print(f"📏 Neighbor tiles shape: {neighbors.shape} | dtype: {neighbors.dtype}")

    # --- Label ---
    label = item['label']
    print(f"🧬 Label shape: {label.shape} | dtype: {label.dtype}")
    assert label.shape[0] == 35 and label.dtype == torch.float32, "❌ Label 應為 float32 且長度為 35"

    # --- Coordinates (meta) ---
    coordinates = item['meta']
    print(f"📌 Coordinates (meta): x = {coordinates[1]}, y = {coordinates[2]}")

    # --- Node Features ---
    if 'node_feat' in item and item['node_feat'] is not None:
        node_feat = item['node_feat']
        print(f"📊 Node feature shape: {node_feat.shape} | dtype: {node_feat.dtype}")


    # --- Edge Features ---
    if 'edge_feat' in item and item['edge_feat'] is not None:
        edge_feat = item['edge_feat']
        print(f"🔗 Edge feature shape: {edge_feat.shape} | dtype: {edge_feat.dtype}")
        print(f"🔗 Sample edge_feat[0]: {edge_feat[0]}")
        assert edge_feat.ndim == 2 and edge_feat.shape[1] == 5, "❌ Edge feature 應為 (k, 5)"
    else:
        print("⚠️ Edge features 未提供")

    # --- Adjacency List ---
    if 'adj_list' in item and item['adj_list'] is not None:
        adj_list = item['adj_list']
        print(f"📎 Adjacency list shape: {adj_list.shape} | dtype: {adj_list.dtype}")
        print(f"📎 Sample: {adj_list[:3]}")
        assert adj_list.ndim == 2 and adj_list.shape[1] == 2, "❌ Adjacency list 應為 (k, 2)"
    else:
        print("⚠️ Adjacency list 未提供")

    print("✅ All checks passed!")




    
check_dataset_item(image_train_dataset)


🔍 Checking dataset sample: 0
📏 Center tile shape: torch.Size([3, 32, 32]) | dtype: torch.float32 | min: 0.451, max: 1.000
📏 Subtiles shape: torch.Size([3, 64, 64]) | dtype: torch.float32
📏 Neighbor tiles shape: torch.Size([3, 128, 128]) | dtype: torch.float32
🧬 Label shape: torch.Size([35]) | dtype: torch.float32
📌 Coordinates (meta): x = 1554, y = 1297
📊 Node feature shape: torch.Size([14]) | dtype: torch.float32
🔗 Edge feature shape: torch.Size([7, 5]) | dtype: torch.float32
🔗 Sample edge_feat[0]: tensor([26.0000, 26.0000,  0.0000,  0.0000,  0.0385])
📎 Adjacency list shape: torch.Size([7, 2]) | dtype: torch.float32
📎 Sample: tensor([[6.2800e+02, 3.8462e-02],
        [2.3700e+02, 3.8462e-02],
        [1.3880e+03, 3.7851e-02]])
✅ All checks passed!


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_30582/3814644018.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'node_feat': torch.tensor(self.node_feats[idx], dtype=torch.float) if self.node_feats is not None else None,


In [15]:
import torch
from torch.utils.data import Subset
import numpy as np

# 確定數據集的總大小（兩者必須相同）
dataset_size = len(image_train_dataset)  # 同時 image_train_dataset 和 graph_train_dataset 長度應相同

# 設定訓練比例，這裡以 80% 為訓練集，20% 為驗證集
train_ratio = 0.8
split_index = int(np.floor(train_ratio * dataset_size))

# 生成隨機索引
indices = torch.randperm(dataset_size).tolist()

# 將索引拆分為訓練和驗證部分
train_indices = indices[:split_index]
val_indices = indices[split_index:]

# 使用 Subset 根據相同的索引建立子集
image_train_subset = Subset(image_train_dataset, train_indices)
image_val_subset = Subset(image_train_dataset, val_indices)

# 現在，你有兩對 DataLoader，且它們的索引是一一對應的
print(f"✅ Train set size: {len(image_train_subset)} samples")
print(f"✅ Validation set size: {len(image_val_subset)} samples")

✅ Train set size: 6679 samples
✅ Validation set size: 1670 samples


In [36]:
from torch.utils.data import Dataset
# from torch.utils.data import DataLoader
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data, Batch



train_loader = DataLoader(image_train_subset, batch_size=1, shuffle=True)

val_loader = DataLoader(image_val_subset, batch_size=1, shuffle=True)


# 測試一下 DataLoader 取出的數據
for batch in train_loader:

    print("Image sample keys:", batch.keys() if hasattr(batch, 'keys') else "Not a dict")
    break

Image sample keys: dict_keys(['S_tile', 'M_tile', 'L_tile', 'label', 'meta', 'normal_coord', 'node_feat', 'adj_list', 'edge_feat'])


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_30582/3814644018.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'node_feat': torch.tensor(self.node_feats[idx], dtype=torch.float) if self.node_feats is not None else None,


Poteintial issues:
# 1. my val_set tiles image may be included in the sub_tiles of train_set

Note: Since neighbor tiles are reused across samples, some mild information overlap may exist between train and val sets. However, final test set is completely held out.

# Neural Network

import torch
import torch.nn as nn
from torch_geometric.nn import GraphSAGE, global_mean_pool
from torch_geometric.data import Data


class MultiLayerAttentionHead(nn.Module):
    def __init__(self, input_dim, output_dim, num_layers=1):
        super(MultiLayerAttentionHead, self).__init__()
        self.attention_layers = nn.ModuleList([
            nn.MultiheadAttention(embed_dim=input_dim, num_heads=1)
            for _ in range(num_layers)
        ])
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        attn_output = x
        for attn_layer in self.attention_layers:
            attn_output, _ = attn_layer(attn_output, attn_output, attn_output)  # Self-attention on the feature vector
        return self.fc(attn_output)


# Define a CNN for each scale (S_tile, M_tile, L_tile)
class SmallScaleCNN(nn.Module):
    def __init__(self):
        super(SmallScaleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.conv(x).view(x.size(0), -1)  # Flatten the output for feature vector

class MediumScaleCNN(nn.Module):
    def __init__(self):
        super(MediumScaleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.conv(x).view(x.size(0), -1)  # Flatten the output for feature vector

class LargeScaleCNN(nn.Module):
    def __init__(self):
        super(LargeScaleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.conv(x).view(x.size(0), -1)  # Flatten the output for feature vector


class GraphSAGEModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphSAGEModel, self).__init__()
        # GraphSAGE layer
        self.graphsage = GraphSAGE(input_dim, hidden_dim, aggr='mean')
        self.fc = nn.Linear(hidden_dim, output_dim)  # For predicting 35 cell types

    def forward(self, data):
        # Assuming `data` contains the features and edge_index for the graph
        x, edge_index = data.x, data.edge_index
        x = self.graphsage(x, edge_index)  # Apply GraphSAGE to the features
        x = global_mean_pool(x, data.batch)  # Global pooling for each graph (slide)
        return self.fc(x)  # Predict the cell type composition


class AttentionHead(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(AttentionHead, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=input_dim, num_heads=1)
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        # Apply attention mechanism
        attn_output, _ = self.attention(x, x, x)  # Self-attention on the feature vector
        return self.fc(attn_output)


class CellTypePredictionModel(nn.Module):
    def __init__(self, small_cnn, medium_cnn, large_cnn, graphsage_model, attention_head):
        super(CellTypePredictionModel, self).__init__()
        self.small_cnn = small_cnn  # CNN for small-scale patch
        self.medium_cnn = medium_cnn  # CNN for medium-scale patch
        self.large_cnn = large_cnn  # CNN for large-scale patch
        self.graphsage_model = graphsage_model  # GraphSAGE for graph-based feature learning
        self.attention_head = attention_head  # Attention mechanism to focus on important features

    def forward(self, S_tile, M_tile, L_tile, edge_index, normal_coords):
        # Step 1: CNN feature extraction for each scale (S_tile, M_tile, L_tile)
        S_features = self.small_cnn(S_tile)
        M_features = self.medium_cnn(M_tile)
        L_features = self.large_cnn(L_tile)
        
        # Combine features from all scales into one feature vector (F_i)
        combined_features = torch.cat([S_features, M_features, L_features], dim=1)
        
        # Step 2: Create PyG Data object for Graph Neural Network
        data = Data(x=combined_features, edge_index=edge_index, pos=normal_coords)
        
        # Step 3: GraphSAGE layer to learn spatial context-aware features
        graphsage_output = self.graphsage_model(data)
        
        # Step 4: Attention mechanism to focus on the relevant features for cell-type prediction
        attention_output = self.attention_head(graphsage_output.unsqueeze(0))  # Add batch dimension
        return attention_output.squeeze(0)  # Remove batch dimension for final output



# Instantiate the individual CNNs for each scale
small_cnn = SmallScaleCNN()
medium_cnn = MediumScaleCNN()
large_cnn = LargeScaleCNN()

# Instantiate the GraphSAGE model and attention head
graphsage_model = GraphSAGEModel(input_dim=16 + 32 + 64, hidden_dim=256, output_dim=256)
attention_head = AttentionHead(input_dim=256, output_dim=35)  # Predict 35 cell types

# Instantiate the final model
model = CellTypePredictionModel(small_cnn, medium_cnn, large_cnn, graphsage_model, attention_head)

# Example input: Assuming you have a batch of data
S_tile = torch.rand(32, 3, 32, 32)  # Example batch of small-scale image tiles
M_tile = torch.rand(32, 3, 64, 64)  # Example batch of medium-scale image tiles
L_tile = torch.rand(32, 3, 128, 128)  # Example batch of large-scale image tiles
meta_info = [(f"S_{i}", torch.rand(2)) for i in range(32)]  # Dummy metadata for batch
edge_index = torch.randint(0, 32, (2, 50))  # Dummy graph edge indices for each slide

# Forward pass
output = model(S_tile, M_tile, L_tile, meta_info, edge_index)
print("Output shape:", output.shape)  # Should be (batch_size, 35) for the 35 cell types


In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch_geometric.nn import global_mean_pool

# --- MultiScaleCNN 定義 ---
class MultiScaleCNN(nn.Module):
    def __init__(self):
        super(MultiScaleCNN, self).__init__()
        # S_branch: Input: [3,32,32] → Output: [16,8,8]
        self.conv_s = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),     # [16, 32, 32]
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),           # [16, 16, 16]
            nn.Conv2d(16, 16, kernel_size=3, padding=1),     # [16, 16, 16]
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)            # [16, 8, 8]
        )
        
        # M_branch: Input: [3,64,64] → Output: [32,8,8]
        self.conv_m = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),      # [32, 64, 64]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),           # [32, 32, 32]
            nn.Conv2d(32, 32, kernel_size=3, padding=1),     # [32, 32, 32]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),           # [32, 16, 16]
            nn.Conv2d(32, 32, kernel_size=3, padding=1),     # [32, 16, 16]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)            # [32, 8, 8]
        )
        
        # L_branch: Input: [3,128,128] → Output: [64,8,8]
        self.conv_l = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),      # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),           # [64, 64, 64]
            nn.Conv2d(64, 64, kernel_size=3, padding=1),     # [64, 64, 64]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),           # [64, 32, 32]
            nn.Conv2d(64, 64, kernel_size=3, padding=1),     # [64, 32, 32]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),           # [64, 16, 16]
            nn.Conv2d(64, 64, kernel_size=3, padding=1),     # [64, 16, 16]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)            # [64, 8, 8]
        )
    def forward(self, S_tile, M_tile, L_tile):
        # S_branch: flatten [batch, 16, 8, 8] → [batch, 16*8*8] = [batch, 1024]
        f_s = self.conv_s(S_tile).view(S_tile.size(0), -1)
        # M_branch: flatten [batch, 32, 8, 8] → [batch, 32*8*8] = [batch, 2048]
        f_m = self.conv_m(M_tile).view(M_tile.size(0), -1)
        # L_branch: flatten [batch, 64, 8, 8] → [batch, 64*8*8] = [batch, 4096]
        f_l = self.conv_l(L_tile).view(L_tile.size(0), -1)
        # 拼接三個尺度的特徵
        return torch.cat([f_s, f_m, f_l], dim=1)
    
# --- CustomGAT 定義 ---
class CustomGAT(nn.Module):
    def __init__(self, in_features, n_heads=8, head_dim=32):
        """
        in_features: 每個節點初始特徵維度（這裡應為 14）
        n_heads: 注意力頭數（例如 8）
        head_dim: 每個注意力頭輸出維度（例如 32）
        """
        super(CustomGAT, self).__init__()
        self.n_heads = n_heads
        self.head_dim = head_dim
        # 對每個注意力頭都有一個線性變換矩陣，輸出 shape: [B, head_dim]
        self.W = nn.Parameter(torch.randn(n_heads, in_features, head_dim))
        # 注意力計算參數，分別對 query 和 key 作用
        self.a1 = nn.Parameter(torch.randn(n_heads, head_dim, 1))
        self.a2 = nn.Parameter(torch.randn(n_heads, head_dim, 1))
        self.leakyrelu = nn.LeakyReLU(0.01)
    
    def forward(self, node_features, adj_list, edge_feat):
        """
        node_features: tensor, shape [B, in_features] ；B = batch size（代表 B 個 spots）
        adj_list: tensor, shape [B, num_neighbors, 2]
                  假設每個元素的第一個值是鄰居索引（可能為 float），第二個值為權重（可忽略）
        edge_feat: tensor, shape [B, num_neighbors, edge_feat_dim]；例如 edge_feat_dim = 5
        """
        B = node_features.size(0)
        num_neighbors = adj_list.size(1)
        head_outputs = []
        
        for h in range(self.n_heads):
            # 線性變換：對每個節點做變換
            # transformed: [B, head_dim]
            transformed = torch.matmul(node_features, self.W[h])
            # 初始化注意力得分矩陣 e，形狀 [B, B]，但我們只對每個 node i 與其鄰居 j 進行計算
            # 這裡我們用 -inf 填充非鄰居位置
            e = torch.full((B, B), float('-inf'), device=node_features.device)
            
            # 對每個節點 i
            for i in range(B):
                # 取出該 node 的鄰居資訊，形狀 [num_neighbors, 2]
                # 我們需要獲得鄰居索引，並確保為整數
                neighbors = adj_list[i, :, 0]  # shape: [num_neighbors]
                # 如果 neighbors 是浮點型，將其轉換為 int
                neighbors = neighbors.long()  # 如果原始存的是 float，可直接用 .long()
                for k in range(num_neighbors):
                    j = int(neighbors[k].item())  # j 為鄰居索引
                    # 取得變換後的特徵：query 與 key 分別 [head_dim, 1]
                    query = transformed[i].unsqueeze(1)
                    key   = transformed[j].unsqueeze(1)
                    e_ij = torch.matmul(self.a1[h].T, query) + torch.matmul(self.a2[h].T, key)
                    e_ij = self.leakyrelu(e_ij)
                    # 如果提供了 edge_feat，根據邊特徵調整 e_ij
                    if edge_feat is not None:
                        # 取出對應邊的特徵，形狀: [edge_feat_dim]
                        current_edge_feat = edge_feat[i, k]  # tensor, shape: [edge_feat_dim]
                        # 這裡簡單使用第一個值作為調整因子
                        e_ij *= 1.0 / (current_edge_feat[0] + 1e-6)
                    # 將計算好的 e_ij 存入對應的位置 [i, j]
                    e[i, j] = e_ij.squeeze()
            
            # 對每個節點的鄰居注意力得分進行 softmax
            attention = F.softmax(e, dim=1)  # [B, B]
            # 聚合鄰居的訊息：對每個節點 i，用鄰居 j 的變換後特徵加權求和
            head_output = torch.zeros(B, self.head_dim, device=node_features.device)
            for i in range(B):
                weighted_sum = torch.zeros(self.head_dim, device=node_features.device)
                for j in range(B):
                    if attention[i, j] > 0:
                        weighted_sum += attention[i, j] * transformed[j]
                head_output[i] = F.relu(weighted_sum)
            head_outputs.append(head_output)
        
        # 多頭拼接：最終輸出 shape [B, n_heads*head_dim]
        multi_head_output = torch.cat(head_outputs, dim=1)
        return multi_head_output
# --- HEVisium 模型 ---
class HEVisium(nn.Module):
    def __init__(self, cnn_backbone, custom_gat, final_out_dim, cnn_out_dim, gat_out_dim):
        super(HEVisium, self).__init__()
        self.cnn_backbone = cnn_backbone  # 用於圖像特徵提取
        self.custom_gat = custom_gat      # 用於圖結構特徵提取
        self.fc = nn.Linear(cnn_out_dim + gat_out_dim, final_out_dim)
    
    def forward(self, S_tile, M_tile, L_tile, node_features, adj_lists, edge_features):
        # CNN 部分，提取 image-based 特徵
        cnn_features = self.cnn_backbone(S_tile, M_tile, L_tile)  # shape: [N, cnn_out_dim]
        # GAT 部分，基於 engineered node features（維度 14）、adj_lists 與 edge_features
        gat_embedding = self.custom_gat(node_features, adj_lists, edge_features)  # [N, gat_out_dim]
        combined = torch.cat([cnn_features, gat_embedding], dim=1)
        out = self.fc(combined)
        return out




In [38]:

for batch in train_loader:
    
    print("train_loader['adj_list'] shape:", batch['adj_list'].shape)
    print("train_loader['node_feat'] shape:", batch['node_feat'].shape)
    print("train_loader['edge_feat'] shape:", batch['edge_feat'].shape)

    break

train_loader['adj_list'] shape: torch.Size([1, 7, 2])
train_loader['node_feat'] shape: torch.Size([1, 14])
train_loader['edge_feat'] shape: torch.Size([1, 7, 5])


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_30582/3814644018.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'node_feat': torch.tensor(self.node_feats[idx], dtype=torch.float) if self.node_feats is not None else None,


In [45]:
# --- 使用示例 ---
if __name__ == "__main__":
    # 假設我們有以下數據，對應 spot 級別
    N = 100  # 100 spots
    # 圖像 patch 數據（來自 CNN 部分），每個尺寸分別如下：
    S_tile = torch.randn(N, 3, 32, 32)
    M_tile = torch.randn(N, 3, 64, 64)
    L_tile = torch.randn(N, 3, 128, 128)
    # engineered node features：假設每個 spot 為 256 維
    node_features = torch.randn(N, 14)
    # 邊信息：adjacency_lists 為每個 spot 的鄰接列表，這裡模擬
    adj_lists = [[(j, 1.0) for j in range(N) if j != i][:15] for i in range(N)]
    # 邊特徵：假設每條邊用 5 維特徵表示
    edge_features = [[np.random.rand(5) for _ in range(15)] for i in range(N)]
    
    final_out_dim = 35  # 預測 35 個細胞類型的 abundance
edge_features = np.array(edge_features)
edge_features.shape

(100, 15, 5)

In [28]:
import torch
import numpy as np
from scipy.stats import spearmanr
from tqdm import tqdm

# 🧠 訓練一個 epoch
def train_one_epoch(model, dataloader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    pbar = tqdm(dataloader, desc="Training", leave=False)
    
    for batch in pbar:
        
        S_tile, M_tile, L_tile = batch['S_tile'].to(device), batch['M_tile'].to(device), batch['L_tile'].to(device)
        label = batch['label'].to(device)
        node_feat = batch['node_feat'].to(device)
        adj_list = batch['adj_list'].to(device)
        edge_feat = batch['edge_feat'].to(device)
        # 優化器清空梯度
        optimizer.zero_grad()
        
        # 前向傳播：計算預測結果
        out = model(S_tile, M_tile, L_tile, node_feat, adj_list, edge_feat)

        # 計算損失
        loss = loss_fn(out, label)
        loss.backward()  # 反向傳播
        optimizer.step()  # 更新模型參數

        total_loss += loss.item() * S_tile.size(0)
        avg_loss = total_loss / ((pbar.n + 1) * dataloader.batch_size)
        pbar.set_postfix(loss=loss.item(), avg=avg_loss)

    return total_loss / len(dataloader.dataset)


# 📏 驗證模型
def evaluate(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0
    preds, targets = [], []

    pbar = tqdm(dataloader, desc="Evaluating", leave=False)

    with torch.no_grad():
        for batch in pbar:
            # 獲取驗證數據
            S_tile, M_tile, L_tile = batch['S_tile'].to(device), batch['M_tile'].to(device), batch['L_tile'].to(device)
            label = batch['label'].to(device)
            node_feat = batch['node_feat'].to(device)
            adj_list = batch['adj_list'].to(device)
            edge_feat = batch['edge_feat'].to(device)
            # 優化器清空梯度
            optimizer.zero_grad()
            
            # 前向傳播：計算預測結果
            out = model(S_tile, M_tile, L_tile, node_feat, adj_list, edge_feat)


            total_loss += loss.item() * S_tile.size(0)
            preds.append(out.cpu())
            targets.append(label.cpu())

            pbar.set_postfix(loss=loss.item())

    preds = torch.cat(preds).numpy()
    targets = torch.cat(targets).numpy()

    # 使用 Spearman 相關性來計算預測的準確性
    scores = [spearmanr(preds[:, i], targets[:, i])[0] for i in range(preds.shape[1])]
    spearman_avg = np.nanmean(scores)

    return total_loss / len(dataloader.dataset), spearman_avg



    # ✅ Spearman correlation for each gene
    scores = [spearmanr(preds[:, i], targets[:, i])[0] for i in range(preds.shape[1])]
    spearman_avg = np.nanmean(scores)

    return total_loss / len(dataloader.dataset), spearman_avg

# 🔮 預測
def predict(model, dataloader, device):
    model.eval()
    all_preds = []
    all_meta = []

    with torch.no_grad():
        for batch in dataloader:
            # 獲取測試數據
            S_tile, M_tile, L_tile = batch['S_tile'].to(device), batch['M_tile'].to(device), batch['L_tile'].to(device)
            label = batch['label'].to(device)
            node_feat = batch['node_feat'].to(device)
            adj_list = batch['adj_list'].to(device)
            edge_feat = batch['edge_feat'].to(device)
            # 優化器清空梯度
            optimizer.zero_grad()
            
            # 前向傳播：計算預測結果
            out = model(S_tile, M_tile, L_tile, node_feat, adj_list, edge_feat)

            all_preds.append(out.cpu())
            all_meta.extend(batch['meta'])  # 用來存儲測試集的meta信息

    return torch.cat(all_preds).numpy(), all_meta



# callback

In [26]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=True):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.verbose = verbose

    def __call__(self, val_loss):
        if self.best_score is None or val_loss < self.best_score:
            self.best_score = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True


import matplotlib.pyplot as plt
from IPython.display import clear_output


def plot_losses(train_losses, val_losses):
    clear_output(wait=True)
    plt.figure(figsize=(8, 4))
    plt.plot(train_losses, label="Train Loss", marker='o')
    plt.plot(val_losses, label="Val Loss", marker='o')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.title("Training vs Validation Loss")
    plt.show()
# 收集資料

In [37]:
import torch
import csv
import matplotlib.pyplot as plt
from IPython.display import clear_output
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

# 🔧 設定裝置
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"✅ Using device: {device}")

# 🔧 初始化模型 & 優化器
# 🔧 初始化模型 & 優化器
cnn_backbone = MultiScaleCNN()  # 這是你定義的多尺度CNN
#attention_head = MultiLayerAttentionHead(input_dim=256, output_dim=35)  # 假設的 AttentionHead
 
in_features = 14  # 必須和 node_features 的第二個維度一致

    # 創建 CustomGAT 模型，這裡假設仍使用 8 個 heads，每個 head 32 維，那麼輸出維度為 8*32 = 256
custom_gat = CustomGAT(in_features=in_features, n_heads=8, head_dim=32)
    
    # 動態計算 cnn_out_dim：
dummy_output = cnn_backbone(torch.randn(1,3,32,32), torch.randn(1,3,64,64), torch.randn(1,3,128,128))
cnn_out_dim = dummy_output.shape[1]
print("Calculated CNN output dimension:", cnn_out_dim)
    
# 預設 gat_out_dim = 8*32 = 256
gat_out_dim = 256

final_out_dim = 35  # 預測 35 個細胞類型的 abundance

model = HEVisium(cnn_backbone=cnn_backbone, custom_gat=custom_gat,
                     final_out_dim=final_out_dim,
                     cnn_out_dim=cnn_out_dim, gat_out_dim=gat_out_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
loss_fn = nn.MSELoss()  # 或者使用其他損失函數，例如 spearman_loss
#loss_fn = spearman_loss

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
early_stopper = EarlyStopping(patience=10)

# 🔧 儲存 log 的設定
log_file = open("training_log.csv", mode="w", newline="")
csv_writer = csv.writer(log_file)
csv_writer.writerow(["Epoch", "Train Loss", "Val Loss", "Val Spearman", "Learning Rate"])

# 🔧 用來畫圖
train_losses = []
val_losses = []


# 🔁 開始訓練
num_epochs = 150
best_val_loss = float('inf')

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, loss_fn, device)
    val_loss, val_spearman = evaluate(model, val_loader, loss_fn, device)

    # ✅ 儲存最好的模型
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pt")
        print("✅ Saved best model!")

    # ✅ 調整學習率
    scheduler.step(val_loss)

    # ✅ 寫入 CSV log
    lr = optimizer.param_groups[0]['lr']
    csv_writer.writerow([epoch+1, train_loss, val_loss, val_spearman, lr])

    # ✅ 印 epoch 結果
    print(f"Epoch {epoch+1}/{num_epochs} | loss: {train_loss:.4f} | val_loss: {val_loss:.4f} | ρ: {val_spearman:.4f} | lr: {lr:.2e}")

    # ✅ 更新 loss list 並畫圖
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    plot_losses(train_losses, val_losses)

    # ✅ Early stopping
    early_stopper(val_loss)
    if early_stopper.early_stop:
        print("⛔ Early stopping triggered.")
        break

# ✅ 關閉 log 檔案
log_file.close()


✅ Using device: mps
Calculated CNN output dimension: 7168


Training:   0%|          | 0/6679 [00:00<?, ?it/s]/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_30582/3814644018.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'node_feat': torch.tensor(self.node_feats[idx], dtype=torch.float) if self.node_feats is not None else None,


IndexError: index 1292 is out of bounds for dimension 0 with size 1

In [113]:
# 測試 train_loader 是否返回正確格式的數據
for batch in train_loader:
    print(f"Batch type: {type(batch)}")
    if isinstance(batch, Data):
        print("✅ This batch is a Data object!")
        print(f"Batch keys: {batch.keys()}")
        
        # 打印 S_tile 的 shape 來檢查
        if 'S_tile' in batch:
            print(f"S_tile shape: {batch['S_tile'].shape}")
        else:
            print("⚠️ S_tile not found in batch!")
        
    else:
        print("⚠️ This batch is not a Data object")
    break  # 只檢查第一個批次


Batch type: <class 'abc.DataBatch'>
✅ This batch is a Data object!
Batch keys: ['L_tile', 'label', 'S_tile', 'ptr', 'M_tile', 'edge_index', 'meta', 'normal_coord', 'batch']
S_tile shape: torch.Size([96, 32, 32])


＃# Prediction

In [59]:
# ===== 需要的 Libraries =====
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
import csv

# ===== 載入訓練好的模型權重 =====
from hevisum_model import HEVisumModel
from hevisum_dataset import importDataset

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"✅ Using device: {device}")

model = VisionMLPModelWithCoord().to(device)
model.load_state_dict(torch.load("best_model.pt"))
model.eval()


✅ Using device: mps


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_54542/3135847424.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model

VisionMLPModelWithCoord(
  (encoder_spot): CNNEncoder(
    (cnn): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): AdaptiveAvgPool2d(output_size=(1, 1))
    )
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (linear): Linear(in_features=64, out_features=64, bias=True)
  )
  (encoder_subtiles

In [61]:
# 正確方式
test_data = torch.load("test_dataset.pt")

raw_coords = []
for _meta in data['meta_info']:
    if _meta is not None:
        _, x, y = _meta
    else:
        x, y = 0, 0
    raw_coords.append([x, y])

raw_coords = np.array(raw_coords)


coord_scaler = StandardScaler()
normalized_coords = coord_scaler.fit_transform(raw_coords)

# ====== Step 2: 建立 Dataset ======
test_dataset = importDataset(
    center_tile=test_data['tiles'],
    subtiles=test_data['subtiles'],
    neighbor_tiles=test_data['neighbor_tiles'],
    label=np.zeros((len(test_data['tiles']), 35)),  # dummy label
    meta=normalized_coords
)


test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_54542/1343659088.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_data = torch.load("test_dataset.pt")


In [62]:
test_preds, test_meta = predict(model, test_loader, device)


coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch

In [ ]:
test_preds

array([[0.29158217, 0.12338457, 0.17179191, ..., 0.03849616, 0.01655579,
        0.02754541],
       [0.29158217, 0.12338457, 0.17179191, ..., 0.03849616, 0.01655579,
        0.02754541],
       [0.25503415, 0.13337179, 0.1524582 , ..., 0.04139816, 0.01808124,
        0.03039141],
       ...,
       [0.29158217, 0.12338457, 0.17179191, ..., 0.03849616, 0.01655579,
        0.02754541],
       [0.29158217, 0.12338457, 0.17179191, ..., 0.03849616, 0.01655579,
        0.02754541],
       [0.23385419, 0.13244098, 0.13941698, ..., 0.04153137, 0.01910294,
        0.03005139]], dtype=float32)

In [66]:
import h5py

# ==== 讀取 test spot index 用於對應 ID ====
with h5py.File("./elucidata_ai_challenge_data.h5", "r") as f:
    test_spots = f["spots/Test"]
    test_spot_table = pd.DataFrame(np.array(test_spots['S_7']))  # Example: S_7


ensemble_df = pd.DataFrame(test_preds, columns=[f"C{i+1}" for i in range(test_preds.shape[1])])
ensemble_df.insert(0, 'ID', test_spot_table.index)
ensemble_df.to_csv("submission.csv", index=False)
print("✅ Saved submission.csv")


✅ Saved submission.csv
